In [1]:
# Some tests of XSM and NuSTAR lightcurves in comparison to other X-ray data
# 
# 18-04-2023    IGH

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy import timeseries as ts
from sunpy.time import parse_time
from astropy.io import fits
import astropy.time as atime
import astropy.units as u
import datetime
import pandas as pd
import os
from pathlib import Path
import glob
from stixdcpy.quicklook import LightCurves
from stixdcpy.net import Request as jreq

import nsx_func

import warnings
warnings.simplefilter('ignore')

plt.rcParams.update({'font.size': 18,'font.family':"sans-serif",\
            'font.sans-serif':"Arial",'mathtext.default':"regular"})

In [3]:
# List of all the NuSTAR observations, with my naming, 
# based off first day of observing per campaign
dobs=['20140910','20141101','20141211',
    '20150429','20150901',
    '20160219','20160422','20160726',
    '20170321','20170821','20170911','20171010',
    '20180529','20180907','20180928',
    '20190112','20190412','20190425','20190702',
    '20200129','20200221','20200606','20200912',
    '20210108','20210429','20210720','20210730','20211117',
    '20220224','20220603','20220906','20221209',
    '20230318']

# Data directory
# ddir=str(Path.home())+'/data/heasarc_nustar/'
ddir='/Volumes/Samsung_T5/data/heasarc_nustar/'

In [4]:
# Find all pointings/IDs for the chosen observation 
for icamp in [24,28,30,32]:
    maindir=ddir+'ns_'+dobs[icamp]+'/'
    print(maindir)
    # Most start with 20 or 90?
    ids = [f.name for f in os.scandir(maindir) \
        if (f.is_dir() and (f.name.startswith('20') or f.name.startswith('80') or f.name.startswith('90')))]
    ids=sorted(ids)
    print(ids)


/Volumes/Samsung_T5/data/heasarc_nustar/ns_20210429/
['20615001001', '20615002001', '20615003001', '20615004001', '20615005001', '20616001001', '20616002001', '20616003001', '20616004001', '20616005001', '20617001001', '20617002001', '20617003001', '20617004001', '20617005001', '20617006001']
/Volumes/Samsung_T5/data/heasarc_nustar/ns_20220224/
['20621001001', '20621002001', '20621003001', '20621004001', '20622001001', '20622002001', '20622003001', '20622004001', '20623001001', '20623002001', '20623003001', '20623004001']
/Volumes/Samsung_T5/data/heasarc_nustar/ns_20220906/
['90810201001', '90810202001', '90810203001', '90810204001']
/Volumes/Samsung_T5/data/heasarc_nustar/ns_20230318/
['20801021001', '20801022001', '20801023001', '20801024001', '20801025001', '20801026001', '20801027001']


In [5]:
# Do all of 2022 and 2023 (so far) but not June (as mosaics and don't have locally)
gdir=str(Path.home())+'/data/goes_xrs/'
xsmdir=str(Path.home())+'/data/xsm/'

for icamp in [24,28,30,32]:
# for icamp in [28]:
    maindir=ddir+'ns_'+dobs[icamp]+'/'
    print(icamp, ' --- ',maindir)
    # Most start with 20 or 80 or 90?
    ids = [f.name for f in os.scandir(maindir) \
        if (f.is_dir() and (f.name.startswith('20') or f.name.startswith('80') or f.name.startswith('90')))]
    ids=sorted(ids)
    # Filter out random other dirs
    ids=[f for f in ids if not f.endswith('_new')]
    # Remove the mosiac from 20220603
    if (dobs[icamp] == '20220603'):
        ids=[f for f in ids if f.startswith('2062400')]      
    print("#: ",len(ids),' --- ',ids)

    # # STIX + GOES/XRS + NuSTAR  lightcurves
    for nsid in ids:
        print(nsid)

        df_test=nsx_func.nsrate(maindir=maindir,nsid=nsid,lvt=True,englow=2,enghigh=10)
        df10 = df_test.resample('10s', level=0).mean()
        lla=df10["lvta"].values
        lla=lla[lla>0]
        llb=df10["lvtb"].values
        llb=llb[llb>0]
        lvta=f'({100*np.min(lla):.2f} - {100*np.max(lla):.2f}%)'
        lvtb=f'({100*np.min(llb):.2f} - {100*np.max(llb):.2f}%)'
        # yrange for NuSTAR plot
        alldata=np.concatenate((df10["rta"].values,df10["rtb"].values))
        alldata=alldata[alldata > 0]
        maxy=1.4*max(alldata)
        miny=0.98*min(alldata)
        yr=[miny,maxy]
        #  Get it per detector quadrant
        dft0=nsx_func.nsrate(maindir=maindir,nsid=nsid,lvt=False,englow=2,enghigh=10,det_id=[0])
        df0= dft0.resample('10s', level=0).mean()
        dft1=nsx_func.nsrate(maindir=maindir,nsid=nsid,lvt=False,englow=2,enghigh=10,det_id=[1])
        df1= dft1.resample('10s', level=0).mean()
        dft2=nsx_func.nsrate(maindir=maindir,nsid=nsid,lvt=False,englow=2,enghigh=10,det_id=[2])
        df2= dft2.resample('10s', level=0).mean()
        dft3=nsx_func.nsrate(maindir=maindir,nsid=nsid,lvt=False,englow=2,enghigh=10,det_id=[3])
        df3= dft3.resample('10s', level=0).mean()
        alldet=np.concatenate((df0["rta"].values,df1["rta"].values,df2["rta"].values,df3["rta"].values))
        yrdet=[0.98*min(alldet),1.3*max(alldet)]

        # Time range to nearest 5mins
        mint=df10.index[0].to_pydatetime()
        mint-=datetime.timedelta(minutes=5,seconds=mint.second)
        mint-=datetime.timedelta(minutes=mint.minute % 5)
        maxt=df10.index[-1].to_pydatetime()
        maxt+=datetime.timedelta(minutes=5,seconds=60-maxt.second)
        maxt-=datetime.timedelta(minutes=maxt.minute % 5)
        tr=[mint,maxt]

        # Get the GOES XRS data
        gtimf=np.unique([f'{mint:%Y%m%d}',f'{maxt:%Y%m%d}'])
        gfiles=[glob.glob(gdir+'*'+g+'*')[0] for g in gtimf]
        trange=a.Time(tr[0],tr[1])
        gxrs = ts.TimeSeries(gfiles, concatenate=True)
        gxrst=gxrs.truncate(trange.start.iso,trange.end.iso)
        tg_tims=gxrst.index
        tg_x05=gxrst.quantity("xrsa").value
        tg_x18=gxrst.quantity("xrsb").value

        # Get the XSM
        # For initial test just do day time range start on/ignoring past midnight
        year=f'{mint:%Y}'
        mon=f'{mint:%m}'
        day=f'{mint:%d}'

        #  First 5 orbits are 29-Apr 20615*, then 5 on 03-May 20616*, then 6 on 07-May 20617*
        #  Get the 29-Apr data file
        xsmfname=f'{xsmdir}data/{year}/{mon}/{day}/calibrated/ch2_xsm_{year}{mon}{day}_v1_level2.lc'
        hdulist = fits.open(xsmfname)
        xdata=hdulist[1].data
        xhdr=hdulist[1].header
        hdulist.close()
        mjdref=atime.Time(xhdr['mjdref'],format='mjd')
        xtims=atime.Time(mjdref+xdata['time']*u.s,format='mjd')
        xdf=pd.DataFrame(xdata["rate"],index=xtims.datetime, columns=['rate'])
        xdf_tr=xdf.truncate(tr[0],tr[1])
        xdfdata=xdf_tr["rate"].values

        # Anything in the GOES flare list? Obviously needs the internet to work
        gsflin = Fido.search(trange,a.hek.EventType("FL"),a.hek.FRM.Name == "SSW Latest Events")
        gsflhk=gsflin["hek"]
        # Anything from AIA?
        aiaflin = Fido.search(trange,a.hek.EventType("FL"),a.hek.FRM.Name =="Flare Detective - Trigger Module")
        aiaflhk=aiaflin["hek"]   

        # Control the time axis labelling
        myFmt = matplotlib.dates.DateFormatter('%H:%M')
        majorx= matplotlib.dates.MinuteLocator(interval=10)
        minorx= matplotlib.dates.MinuteLocator(interval=1)
        plt.rcParams.update({'font.size': 16})
        fig, axs= plt.subplots(4,figsize=(10, 12))
        
        axs[0].plot(xdf_tr.index, xdfdata,drawstyle='steps-post',marker=None,color='goldenrod',lw=1,label='XSM')
        axs[0].set_ylabel('XSM [counts]')
        axs[0].set_ylim([0.98*np.min(xdfdata),1.3*np.max(xdfdata)])
        axs[0].legend(loc=2,fontsize=14)

        axs[1].plot(tg_tims,tg_x18,drawstyle='steps-post',\
            marker=None,color='firebrick',lw=2,label='G16 $1-8\;\AA$')
        axs[1].set_ylabel('XRS [W$\;m^{-2}$]')
        # Only set limits if not all missing/nans
        if not np.isnan(np.nanmin(tg_x18)):
            yrgs=[0.95*np.nanmin(tg_x18),1.3*np.nanmax(tg_x18)]
        else:
            yrgs=[0.5e-7,3.5e-7]
        mngs=np.mean(yrgs)
        gspow = int(f"{mngs:e}".split('e')[1])
        mngs=round(mngs/10**gspow)*10**gspow
        match gspow:
            case -8:
                gscls="A"
            case -7:
                gscls="B"
            case -6:
                gscls="C"
            case -5:
                gscls="M"
        if (mngs < yrgs[0]):   
            yrgs[0] = mngs
        if (mngs > yrgs[1]):   
            yrgs[1] = mngs
        axs[1].set_ylim(yrgs)
        axs[1].plot(tr,[mngs,mngs],linestyle='dotted',color='grey')
        axs[1].annotate(f' {gscls}{mngs/10**gspow:.0f}',(tr[1],mngs),
                        color='grey',fontweight="bold")

        for gg in gsflhk:
            tfl=a.Time(gg["event_starttime"],gg["event_endtime"])
            ptim=parse_time(gg["event_peaktime"]).datetime
            axs[1].axvspan(tfl.start.datetime,tfl.end.datetime,color='silver',alpha=0.4, \
                label=gg["fl_goescls"]+f' ({ptim:%H:%M})')
            # for ax in axs:
            #     ax.axvline(ptim,color='black',ls='-.')
            axs[1].axvline(ptim,color='black',ls='-.')
            axs[2].axvline(ptim,color='black',ls='-.')
            axs[3].axvline(ptim,color='black',ls='-.')


        for aa in aiaflhk:
            #  Only plot it if peak time during NuSTAR time window
            if (aa["event_peaktime"] >= tr[0]) and (aa["event_peaktime"] <= tr[1]):
                ptim=parse_time(aa["event_peaktime"]).datetime
                axs[1].axvline(ptim,color='darkorchid',ls=':')
                # axs[1].axvline(ptim,color='darkorchid',ls=':',label=f'AF ({ptim:%H:%M})')
                axs[2].axvline(ptim,color='darkorchid',ls=':')
                axs[3].axvline(ptim,color='darkorchid',ls=':')
        axs[1].legend(loc=2,ncols=5,fontsize=14)
        
        axs[2].plot(df0.index,df0["rta"].values,drawstyle='steps-post',lw=2,color='royalblue',label='A0')
        axs[2].plot(df1.index,df1["rta"].values,drawstyle='steps-post',lw=2,color='darkorange',label='A1')
        axs[2].plot(df2.index,df2["rta"].values,drawstyle='steps-post',lw=2,color='lightseagreen',label='A2')
        axs[2].plot(df3.index,df3["rta"].values,drawstyle='steps-post',lw=2,color='orchid',label='A3')

        axs[2].set_ylabel('NuSTAR [count$\;s^{-1}$]')
        axs[2].set_ylim(yrdet)
        axs[2].legend(loc=2,ncols=4,fontsize=14)

        axs[3].plot(df10.index,df10["rta"].values,drawstyle='steps-post',\
            lw=2,color='rebeccapurple',label='A >2keV '+lvta)
        axs[3].plot(df10.index,df10["rtb"].values,drawstyle='steps-post',\
            lw=2,color='teal',label='B >2keV '+lvtb)
        axs[3].set_xlabel(f'Start Time {mint:%d-%b-%Y %H:%M:%S}')
        axs[3].set_ylabel('NuSTAR [count$\;s^{-1}$]')
        axs[3].set_ylim(yr)
        axs[3].legend(loc=2,fontsize=14,ncols=2)
        for ax in axs:
            ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0), useOffset=True)
            ax.set_xlim(tr)
            ax.xaxis.set_major_locator(majorx)
            ax.xaxis.set_minor_locator(minorx)
            ax.xaxis.set_major_formatter(myFmt)
        plt.annotate(nsid,(0.01,0.01),xycoords='figure fraction',fontsize=14)

        fig.subplots_adjust(bottom=0.06, top=0.97, left=0.12, right=0.95)
        plt.savefig('xsm_figs/ns_'+dobs[icamp]+f'/ltc_{mint:%Y%m%d_%H%M}_'+nsid+'_ngx.png')
        # as looping don't need to plot the figures to the notebook
        plt.close()
        # break
    # break


24  ---  /Volumes/Samsung_T5/data/heasarc_nustar/ns_20210429/
#:  16  ---  ['20615001001', '20615002001', '20615003001', '20615004001', '20615005001', '20616001001', '20616002001', '20616003001', '20616004001', '20616005001', '20617001001', '20617002001', '20617003001', '20617004001', '20617005001', '20617006001']
20615001001
20615002001
20615003001
20615004001
20615005001
20616001001
20616002001
20616003001
20616004001
20616005001
20617001001
20617002001
20617003001
20617004001
20617005001
20617006001
28  ---  /Volumes/Samsung_T5/data/heasarc_nustar/ns_20220224/
#:  12  ---  ['20621001001', '20621002001', '20621003001', '20621004001', '20622001001', '20622002001', '20622003001', '20622004001', '20623001001', '20623002001', '20623003001', '20623004001']
20621001001
20621002001
20621003001
20621004001
20622001001
20622002001
20622003001
20622004001
20623001001
20623002001
20623003001
20623004001
30  ---  /Volumes/Samsung_T5/data/heasarc_nustar/ns_20220906/
#:  4  ---  ['90810201001', '9

KeyboardInterrupt: 